In [172]:
import numpy as np
import pandas as pd

In [210]:
data = pd.read_csv('savant_data.csv')
df = data[['pitch_type', 'events', 'description',  'zone', 'balls', 'strikes', \
         'hit_distance_sc', 'launch_speed', 'launch_angle']]

In [174]:
df.head(10)

,pitch_type,events,description,zone,balls,strikes,hit_distance_sc,launch_speed,launch_angle
0,CH,double,hit_into_play,8.0,0,1,118.0,107.8,5.0
1,SL,NaN,called_strike,12.0,0,0,NaN,NaN,NaN
2,SL,field_out,hit_into_play,5.0,2,2,146.0,72.5,62.0
3,FF,NaN,swinging_strike,2.0,2,1,NaN,NaN,NaN
4,FF,NaN,called_strike,1.0,2,0,NaN,NaN,NaN
5,SL,NaN,ball,13.0,1,0,NaN,NaN,NaN
6,SL,NaN,ball,14.0,0,0,NaN,NaN,NaN
7,CH,fielders_choice,hit_into_play,14.0,1,1,9.0,66.4,-9.0
8,SL,NaN,called_strike,9.0,1,0,NaN,NaN,NaN
9,CH,NaN,ball,14.0,0,0,NaN,NaN,NaN


In [211]:



df['pitch_type'] = df['pitch_type'].replace({
    'FF' : 1,
    'SI' : 2,
    'SL' : 3,
    'CH' : 4
})

df['events'] = df['events'].replace({
    np.nan : 3,
    'sac_bunt' : 3,
    'pickoff_1b' : 3,
    'caught_stealing_2b' : 3,

    'single' : 0, # 0 is bad outcomes
    'double' : 0,
    'triple' : 0,
    'home_run' : 0,
    'walk' : 0,
    'hit_by_pitch': 0,
    'sac_fly' : 0,

    'field_out' : 1, #1 is good outcomes
    'fielders_choice': 1,
    'grounded_into_double_play' : 1,
    'force_out' : 1,
    'field_error' : 1,
    'strikeout' : 1

})

df['count'] = df['balls'].astype(str) + '-' + df['strikes'].astype(str)
df['count'] = df['count'].astype('category')
df = df.drop(['balls', 'strikes'], axis=1)


df['hit_distance_sc'] = df['hit_distance_sc'].map({np.nan : 0})
df['launch_speed'] = df['launch_speed'].map({np.nan : 0})
df['launch_angle'] = df['launch_angle'].map({np.nan : 0})


df = df.dropna()
df['zone'] = df['zone'].astype(int)
df['pitch_type'] = df['pitch_type'].astype(int)
df.head(10)



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

,pitch_type,events,description,zone,hit_distance_sc,launch_speed,launch_angle,count
1,3,3,called_strike,12,0.0,0.0,0.0,0-0
3,1,3,swinging_strike,2,0.0,0.0,0.0,2-1
4,1,3,called_strike,1,0.0,0.0,0.0,2-0
5,3,3,ball,13,0.0,0.0,0.0,1-0
6,3,3,ball,14,0.0,0.0,0.0,0-0
8,3,3,called_strike,9,0.0,0.0,0.0,1-0
9,4,3,ball,14,0.0,0.0,0.0,0-0
10,3,1,swinging_strike,13,0.0,0.0,0.0,1-2
11,1,3,ball,11,0.0,0.0,0.0,0-2
12,3,3,swinging_strike,13,0.0,0.0,0.0,0-1


In [212]:
good_descriptions = ['field_out', 'fielders_choice', 'grounded_into_double_play', 'force_out', 'field_error', 'strikeout']

fastballs = [1, 2]
offspeed = [3, 4]

low_strikes = [7, 8, 9]
high_strikes = [1, 2, 3]
low_balls = [13, 14]
high_balls = [11, 12]
high_pitches = [1, 2, 3, 11, 12]
low_pitches = [7, 8, 9, 13, 14]
strikes = [1, 2, 3, 4, 5, 6, 7, 8, 9]
balls = [11, 12, 13, 14]

good_counts = ['0-1', '0-2', '1-1', '1-2', '2-2'] # Ahead in the count
bad_counts = ['1-0', '2-0', '3-0', '2-1', '3-1'] # Behind in the count


In [213]:
# Define an executed pitch function that take in a row and returns 1 if good execution, 0 if bad execution
def executed_pitch(row):
    if row['description'] in good_descriptions:
        return 1
    else: # Not a good outcome, not necessarily a bad outcome
        if row['count'] in good_counts:
            if row['pitch_type'] in fastballs and row['zone'] in [high_pitches, low_pitches]:
                return 1
            elif row['pitch_type'] in offspeed and row['zone'] in low_pitches:
                return 1
            else:
                return 0
        else: # if the count is bad
            if row['zone'] in balls:
                return 0
            else: # if it is a strike
                return 1

In [214]:
df['Executed'] = df.apply(executed_pitch, axis=1)
df.head(10)

,pitch_type,events,description,zone,hit_distance_sc,launch_speed,launch_angle,count,Executed
1,3,3,called_strike,12,0.0,0.0,0.0,0-0,0
3,1,3,swinging_strike,2,0.0,0.0,0.0,2-1,1
4,1,3,called_strike,1,0.0,0.0,0.0,2-0,1
5,3,3,ball,13,0.0,0.0,0.0,1-0,0
6,3,3,ball,14,0.0,0.0,0.0,0-0,0
8,3,3,called_strike,9,0.0,0.0,0.0,1-0,1
9,4,3,ball,14,0.0,0.0,0.0,0-0,0
10,3,1,swinging_strike,13,0.0,0.0,0.0,1-2,1
11,1,3,ball,11,0.0,0.0,0.0,0-2,0
12,3,3,swinging_strike,13,0.0,0.0,0.0,0-1,1


In [215]:
# Determine the percentage of good pitches
print(df['Executed'].sum() / len(df['Executed']))

0.4497840172786177


In [180]:
df['description'].unique()

array(['called_strike', 'swinging_strike', 'ball', 'foul', 'blocked_ball',
       'foul_tip', 'hit_by_pitch', 'swinging_strike_blocked',
       'hit_into_play', 'missed_bunt', 'foul_bunt'], dtype=object)

In [216]:
df['description'] = df['description'].map({
    'hit_by_pitch': 1,
    'called_strike' : 2,
    'swinging_strike' : 3,
    'ball' : 4,
    'foul' : 5,
    'blocked_ball' : 6,
    'foul_tip' : 7,
    'swinging_strike_blocked' : 8,
    'hit_into_play' : 9,
    'missed_bunt' : 10,
    'foul_bunt' : 11,
})
df['count'] = df['count'].map({
    '0-0' : 0,
    '0-1' : 1,
    '0-2' : 2,
    '1-0' : 3,
    '1-1' : 4,
    '1-2' : 5,
    '2-0' : 6,
    '2-1' : 7,
    '2-2' : 8,
    '3-0' : 9,
    '3-1' : 10,
    '3-2' : 11,
})
df['count'] = df['count'].astype(int)

In [217]:
df['count'].unique()

array([ 0,  7,  6,  3,  5,  2,  1, 10,  9, 11,  4,  8])

In [183]:
df.dtypes

pitch_type           int64
events               int64
description          int64
zone                 int64
hit_distance_sc    float64
launch_speed       float64
launch_angle       float64
count                int64
Executed             int64
dtype: object

In [207]:
# Define a few functions to help with the analysis
def find_count(count, df):
    if count == '0-1':
        count = 0
    elif count == '0-2':
        count = 1
    elif count == '1-1':
        count = 2
    elif count == '1-2':
        count = 3
    elif count == '2-2':
        count = 4
    elif count == '1-0':
        count = 5
    elif count == '2-0':
        count = 6
    elif count == '3-0':
        count = 7
    elif count == '2-1':
        count = 8
    elif count == '3-1':
        count = 9
    elif count == '3-2':
        count = 10
    else: # 0-0
        count = 11
    
    return df[df['count'] == count]

def count_transform(count):
    if count == '0-1':
        count = 0
    elif count == '0-2':
        count = 1
    elif count == '1-1':
        count = 2
    elif count == '1-2':
        count = 3
    elif count == '2-2':
        count = 4
    elif count == '1-0':
        count = 5
    elif count == '2-0':
        count = 6
    elif count == '3-0':
        count = 7
    elif count == '2-1':
        count = 8
    elif count == '3-1':
        count = 9
    elif count == '3-2':
        count = 10
    else: # 0-0
        count = 11
    
    return count

def find_count_inverse(count):
    if count == 0:
        count = '0-1'
    elif count == 1:
        count = '0-2'
    elif count == 2:
        count = '1-1'
    elif count == 3:
        count = '1-2'
    elif count == 4:
        count = '2-2'
    elif count == 5:
        count = '1-0'
    elif count == 6:
        count = '2-0'
    elif count == 7:
        count = '3-0'
    elif count == 8:
        count = '2-1'
    elif count == 9:
        count = '3-1'
    elif count == 10:
        count = '3-2'
    else: # 0-0
        count = '0-0'
    return count

def convert_pitch_type(pitch_type): # pitch_type is a string 'FF', 'SI', 'SL', 'CH'
    if pitch_type == 'FF':
        pitch_type = 1
    elif pitch_type == 'SI':
        pitch_type = 2
    elif pitch_type == 'SL':
        pitch_type = 3
    elif pitch_type == 'CH':
        pitch_type = 4

    return pitch_type

In [218]:
known = df[['pitch_type', 'count', 'Executed']] # Executed needed to train the model
# known.to_csv('executed.csv')